[View in Colaboratory](https://colab.research.google.com/github/TRIPTIRATHI/Assignment-4/blob/master/TRIPTI_BATCH_7_ASSIGNMENT_4B.ipynb)

In [0]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential, load_model
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam, SGD

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 90
num_classes = 10
epochs = 50
l = 12
num_filter = 72
compression = 0.5
dropout_rate = 0.3

In [0]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

147595264/170498071 [========================>.....] - ETA: 6s

170500096/170498071 [==============================] - 47s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 72, dropout_rate = 0.3):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 60, dropout_rate = 0.3):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AveragePooling =AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AveragePooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 60
dropout_rate = 0.3
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)


Last_Block = add_denseblock(Second_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [0]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_105 (Conv2D)             (None, 32, 32, 60)   1620        input_3[0][0]                    
__________________________________________________________________________________________________
batch_normalization_104 (BatchN (None, 32, 32, 60)   240         conv2d_105[0][0]                 
__________________________________________________________________________________________________
activation_104 (Activation)     (None, 32, 32, 60)   0           batch_normalization_104[0][0]    
__________________________________________________________________________________________________
conv2d_106

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy'])

In [0]:
# checkpoint
filepath="wh.best.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.1, patience=2, verbose=1, mode='auto', min_delta=0.0001, cooldown=0, min_lr=0)
callbacks_list = [checkpoint, reduce_lr]

In [0]:
model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks_list,
                    validation_data=(x_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 503s 10ms/step - loss: 1.4451 - acc: 0.4863 - val_loss: 2.2892 - val_acc: 0.4235

Epoch 00001: val_acc improved from -inf to 0.42350, saving model to wh.best.hdf5
Epoch 2/50
 4410/50000 [=>............................] - ETA: 6:52 - loss: 1.0755 - acc: 0.6249

50000/50000 [==============================] - 487s 10ms/step - loss: 0.9722 - acc: 0.6568 - val_loss: 1.5935 - val_acc: 0.5700

Epoch 00002: val_acc improved from 0.42350 to 0.57000, saving model to wh.best.hdf5
Epoch 3/50
16380/50000 [========>.....................] - ETA: 5:03 - loss: 0.8021 - acc: 0.7178

50000/50000 [==============================] - 486s 10ms/step - loss: 0.7755 - acc: 0.7275 - val_loss: 1.0686 - val_acc: 0.6855

Epoch 00003: val_acc improved from 0.57000 to 0.68550, saving model to wh.best.hdf5
Epoch 4/50
19710/50000 [==========>...................] - ETA: 4:33 - loss: 0.6657 - acc: 0.7669

50000/50000 [==============================] - 487s 10ms/step - loss: 0.6616 - acc: 0.7688 - val_loss: 1.0376 - val_acc: 0.7056

Epoch 00004: val_acc improved from 0.68550 to 0.70560, saving model to wh.best.hdf5
Epoch 5/50
20610/50000 [===========>..................] - ETA: 4:25 - loss: 0.5902 - acc: 0.7920

50000/50000 [==============================] - 487s 10ms/step - loss: 0.5788 - acc: 0.7963 - val_loss: 0.8829 - val_acc: 0.7386

Epoch 00005: val_acc improved from 0.70560 to 0.73860, saving model to wh.best.hdf5
Epoch 6/50
20880/50000 [===========>..................] - ETA: 4:22 - loss: 0.5239 - acc: 0.8164

...

50000/50000 [==============================] - 487s 10ms/step - loss: 0.1396 - acc: 0.9532 - val_loss: 0.4736 - val_acc: 0.8720

Epoch 00046: val_acc did not improve from 0.87320

Epoch 00046: ReduceLROnPlateau reducing learning rate to 1.0000000664932204e-18.
Epoch 47/50
18540/50000 [==========>...................] - ETA: 4:43 - loss: 0.1374 - acc: 0.9544

50000/50000 [==============================] - 487s 10ms/step - loss: 0.1383 - acc: 0.9546 - val_loss: 0.4736 - val_acc: 0.8720

Epoch 00047: val_acc did not improve from 0.87320
Epoch 48/50
22770/50000 [============>.................] - ETA: 4:05 - loss: 0.1379 - acc: 0.9553

50000/50000 [==============================] - 487s 10ms/step - loss: 0.1379 - acc: 0.9549 - val_loss: 0.4712 - val_acc: 0.8725

Epoch 00048: val_acc did not improve from 0.87320

Epoch 00048: ReduceLROnPlateau reducing learning rate to 1.000000045813705e-19.
Epoch 49/50
18630/50000 [==========>...................] - ETA: 4:43 - loss: 0.1354 - acc: 0.9554

50000/50000 [==============================] - 487s 10ms/step - loss: 0.1386 - acc: 0.9529 - val_loss: 0.4718 - val_acc: 0.8724

Epoch 00049: val_acc did not improve from 0.87320
Epoch 50/50
22770/50000 [============>.................] - ETA: 4:05 - loss: 0.1384 - acc: 0.9540

49770/50000 [============================>.] - ETA: 2s - loss: 0.1412 - acc: 0.9527

In [0]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 41s 4ms/step
Test loss: 0.47160090514421465
Test accuracy: 0.8728


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')